In [2]:
import requests
from requests import Response
aqi_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res: Response = requests.get(aqi_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

from pydantic import BaseModel,Field,field_validator

class Site(BaseModel):
    sna:str
    saera:str
    mday:str
    ar:str
    act:int
    total:int
    available_rent_bikes:int
    latitude:float
    longitude:float
    available_return_bikes:int
    
    @field_validator()
    @classmethod

下載成功


In [ ]:
#sna    sarea    mday    ar    act    total    available_rent_bikes    latitude    longitude    available_return_bikes